In [16]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/Covid Data.csv')

UČITAVANJE PODATAKA

In [17]:
# Prikaz prvih nekoliko redova 
print(df.head())

# Broj redova pre čišćenja
print(f"Početni broj redova: {df.shape[0]}")


   USMER  MEDICAL_UNIT  SEX  PATIENT_TYPE   DATE_DIED  INTUBED  PNEUMONIA  \
0      2             1    1             1  03/05/2020       97          1   
1      2             1    2             1  03/06/2020       97          1   
2      2             1    2             2  09/06/2020        1          2   
3      2             1    1             1  12/06/2020       97          2   
4      2             1    2             1  21/06/2020       97          2   

   AGE  PREGNANT  DIABETES  ...  ASTHMA  INMSUPR  HIPERTENSION  OTHER_DISEASE  \
0   65         2         2  ...       2        2             1              2   
1   72        97         2  ...       2        2             1              2   
2   55        97         1  ...       2        2             2              2   
3   53         2         2  ...       2        2             2              2   
4   68        97         1  ...       2        2             1              2   

   CARDIOVASCULAR  OBESITY  RENAL_CHRONIC  TOBACCO

PROVERA NEDOSTAJUĆIH VREDNOSTI

In [18]:
# Iz dokumentacije dataseta vidimo da 97, 98 i 99 predstavljaju nedostajuće vrednosti
# Ove vrednosti tretiramo kao NaN da bismo videli pravo stanje
# Vodimo računa da za kolonu 'AGE' mogu postojati pacijenti sa ovim vrednostima godina, pa cemo nju zasebno proveriti!
# Zasebno ćemo proveriti i DATE_DIED zbog specifičnog formata

print(f"Kolona 'AGE' ima {df['AGE'].isna().sum()} nedostajućih vrednosti")
print(f"Kolona 'DATE_DIED' ima {df['DATE_DIED'].isnull().sum()} nedostajućih vrednosti")

cols_to_replace = [col for col in df.columns if col not in ['AGE', 'DATE_DIED']]
df[cols_to_replace] = df[cols_to_replace].replace({97: np.nan, 98: np.nan, 99: np.nan})

missing_values = df.isna().sum().sort_values(ascending=False)
missing_values_percent = (missing_values / len(df)) * 100

missing_values_report = pd.DataFrame({
    'Broj_NaN': missing_values,
    'Procenat_NaN': missing_values_percent
})

print("-----IZVEŠTAJ O NEDOSTAJUĆIM VREDNOSTIMA -----")
print(missing_values_report)
# Iz izvestaja vidimo 
# INTUBED i ICU -> Ogroman procenat nedostajućih (preko 80%) i to je zato što je ovde vrednost 97 za sve pacijente koji nisu hospitalizovani (što je većina)
# PREGNANCY -> Oko 50% nedostajućih vrednosti jer se podatak ne odnosi na muškarce
# PNEUMONIA -> Ovde imamo oko 1% nedostajućih vrednosti
# OSTALO -> Zanemarljivo malo nedostajućih vrednosti

Kolona 'AGE' ima 0 nedostajućih vrednosti
Kolona 'DATE_DIED' ima 0 nedostajućih vrednosti
-----IZVEŠTAJ O NEDOSTAJUĆIM VREDNOSTIMA -----
                      Broj_NaN  Procenat_NaN
ICU                     856032     81.637651
INTUBED                 855869     81.622106
PREGNANT                527265     50.283957
PNEUMONIA                16003      1.526166
OTHER_DISEASE             5045      0.481129
INMSUPR                   3404      0.324631
DIABETES                  3338      0.318337
TOBACCO                   3220      0.307083
HIPERTENSION              3104      0.296021
CARDIOVASCULAR            3076      0.293350
OBESITY                   3032      0.289154
RENAL_CHRONIC             3006      0.286675
COPD                      3003      0.286389
ASTHMA                    2979      0.284100
DATE_DIED                    0      0.000000
SEX                          0      0.000000
MEDICAL_UNIT                 0      0.000000
USMER                        0      0.000000
PATIENT_

PROVERA DUPLIKATA

In [19]:
duplicate_count = df.duplicated().sum()
print(f"Broj istih redova u datasetu: {duplicate_count}")

# Iako vidimo veliki broj identičnih zapisa, oni se ne možemo da smatramo duplikatima jer dataset ne sadrži jedinstveni identifikator
# pacijenta na osnovu kog bismo pouzdano mogli da znamo da se radi o dupliranom unosu istog pacijenta. 
# Identitet zapisa ne može se pouzdano utvrditi, pa ćemo pretpostavljati da zapisi predstavljaju različite pojedince sa istim karakteristikama.

Broj istih redova u datasetu: 812049


PROVERA NEKONZISTENTNOSTI PODATAKA

In [20]:
# provera godina starosti
print("-----PROVERA STAROSTI-----")
invalid_age = df[(df['AGE'] < 0)] #ne uključujemo i 0 jer su to bebe i iskrivilo bi starosnu distribuciju
print(f"Broj zapisa sa loše unetim godinama: {len(invalid_age)}")

#PROVERA KATEGORIJSKIH PODATAKA U KOJIMA NEMAMO NEDOSTAJUCIH VREDNOSTI
print(f"\n-----PROVERA KATEGORIJSKIH NENEDOSTAJUĆIH PODATAKA-----\n")
# provera pola - dozvoljene su samo vrednosti 1 i 2
print("-----PROVERA POLA-----")
print(df['SEX'].value_counts())

print("-----PROVERA KATEGORIJA ZA KUĆU I BOLNICU-----")
print(df['PATIENT_TYPE'].value_counts())

print(f"-----------------------------------------\n")


# provera da li su pacijenti koji nisu hospitalizovani bili na intenzivnoj nezi
print(f"\n-----PROVERA HOSPITALIZACIJE I INTENZIVNE NEGE-----")
invalid_icu = df[(df['PATIENT_TYPE'] == 1) & (df['ICU'] == 1)]
invalid_intubed = df[(df['PATIENT_TYPE'] == 1) & (df['INTUBED'] == 1)]

print(f"Neispravni ICU zapisi: {len(invalid_icu)}")
print(f"Neispravni INTUBED zapisi: {len(invalid_intubed)}")

# provera da li imamo trudnih muškaraca
print(f"\n-----PROVERA TRUDNOĆE -----")
invalid_pregnancy_men = df[(df['SEX'] == 2) & (df['PREGNANT'] == 1)]
print(f"Neispravni zapisi o trudnoći muškaraca: {len(invalid_pregnancy_men)}")

invalid_pregnancy_baby = df[(df['AGE'] <= 5) & (df['PREGNANT'] == 1)]
print(f"Neispravni zapisi o trudnoći male dece i beba: {len(invalid_pregnancy_baby)}")

# provera da li imamo beba pušača
print(f"\n-----PROVERA PUŠAČA-----")
invalid_smokers = df[(df['AGE'] == 0) & (df['TOBACCO'] == 1)]
print(f"Neispravni zapisi o pušačima (bebe pušači): {len(invalid_smokers)}")

-----PROVERA STAROSTI-----
Broj zapisa sa loše unetim godinama: 0

-----PROVERA KATEGORIJSKIH NENEDOSTAJUĆIH PODATAKA-----

-----PROVERA POLA-----
SEX
1    525064
2    523511
Name: count, dtype: int64
-----PROVERA KATEGORIJA ZA KUĆU I BOLNICU-----
PATIENT_TYPE
1    848544
2    200031
Name: count, dtype: int64
-----------------------------------------


-----PROVERA HOSPITALIZACIJE I INTENZIVNE NEGE-----
Neispravni ICU zapisi: 0
Neispravni INTUBED zapisi: 0

-----PROVERA TRUDNOĆE -----
Neispravni zapisi o trudnoći muškaraca: 0
Neispravni zapisi o trudnoći male dece i beba: 8

-----PROVERA PUŠAČA-----
Neispravni zapisi o pušačima (bebe pušači): 60


KOREKCIJE NEKONZISTENTNOSTI

In [21]:
# -> bebe ne mogu biti pušači
df.loc[(df['AGE'] == 0) & (df['TOBACCO'] == 1), 'TOBACCO'] = 0

# -> deca ispod 5 godina ne mogu biti trudna
df.loc[(df['AGE'] <= 5) & (df['PREGNANT'] == 1), 'PREGNANT'] = 2

IMPUTACIJA/BRISANJE NEDOSTAJUĆIH VREDNOSTI

In [22]:
#ICU i INTUBED -> Ovde cemo za podatke koji nedostaju za pacijente koji su poslati kući (tj. gde je PATIENT_TYPE = 1)
#pretpostaviti da oni nisu bili na intenzivnoj nezi. Zato ćemo NaN vrednosti kod njih zameniti sa 2 (što znači "NE")

#PREGNANT -> Kod muškaraca (SEX = 2), NaN treba da postane NE

# Logička imputacija na osnovu konteksta
df.loc[(df['PATIENT_TYPE'] == 1) & (df['ICU'].isna()), 'ICU'] = 2
df.loc[(df['PATIENT_TYPE'] == 1) & (df['INTUBED'].isna()), 'INTUBED'] = 2
df.loc[(df['SEX'] == 2) & (df['PREGNANT'].isna()), 'PREGNANT'] = 2


# Sve ostale preostale NaN vrednosti (koje su < 1-2% dataseta) možemo obrisati jer imamo dovoljno podataka, pa imputacija neće 
# značajno uticati na rezultat
df.dropna(subset=['PNEUMONIA', 'OTHER_DISEASE', 'INMSUPR', 'DIABETES', 'TOBACCO', 'HIPERTENSION', 'CARDIOVASCULAR', 'OBESITY', 
'RENAL_CHRONIC', 'COPD','ASTHMA'], inplace=True)

missing_values = df.isna().sum().sort_values(ascending=False)
missing_values_percent = (missing_values / len(df)) * 100

missing_values_report = pd.DataFrame({
    'Broj_NaN': missing_values,
    'Procenat_NaN': missing_values_percent
})

print("-----IZVEŠTAJ O NEDOSTAJUĆIM VREDNOSTIMA -----")
print(missing_values_report)

-----IZVEŠTAJ O NEDOSTAJUĆIM VREDNOSTIMA -----
                      Broj_NaN  Procenat_NaN
PREGNANT                  3175      0.309710
ICU                       2311      0.225430
INTUBED                   2153      0.210018
MEDICAL_UNIT                 0      0.000000
USMER                        0      0.000000
DATE_DIED                    0      0.000000
PATIENT_TYPE                 0      0.000000
SEX                          0      0.000000
PNEUMONIA                    0      0.000000
DIABETES                     0      0.000000
COPD                         0      0.000000
ASTHMA                       0      0.000000
AGE                          0      0.000000
INMSUPR                      0      0.000000
HIPERTENSION                 0      0.000000
CARDIOVASCULAR               0      0.000000
OTHER_DISEASE                0      0.000000
OBESITY                      0      0.000000
RENAL_CHRONIC                0      0.000000
TOBACCO                      0      0.000000
CLASIFFI

In [23]:
# Posto na osnovu rezultata vidimo da kolone PREGNANT, ICU I INTUBED imaju <1% nedostajućih vrendosti, 

# Obrisaćemo preostale redove gde nemamo info o ICU ili INTUBED, 
# a pacijent je hospitalizovan (jer su to ključne kolone za kasnije istraživanje)
df.dropna(subset=['ICU', 'INTUBED'], inplace=True)

# Za trudnocu kod žena koristićemo najčešću vrednost i time zamentiti podatke, a mogli smo ih i obrisati pošto ih ima malo
df.loc[(df['SEX'] == 1) & (df['PREGNANT'].isna()), 'PREGNANT'] = df[df['SEX'] == 1]['PREGNANT'].mode()[0]


missing_values = df.isna().sum().sort_values(ascending=False)
missing_values_percent = (missing_values / len(df)) * 100

missing_values_report = pd.DataFrame({
    'Broj_NaN': missing_values,
    'Procenat_NaN': missing_values_percent
})

print("-----IZVEŠTAJ O NEDOSTAJUĆIM VREDNOSTIMA -----")
print(missing_values_report)

# Broj redova nakon čišćenja
print(f"Broj redova nakon čišćenja podataka: {df.shape[0]}")

# Vidimo da smo nakon čišćenja zadržale oko 97.5% početnih podataka


-----IZVEŠTAJ O NEDOSTAJUĆIM VREDNOSTIMA -----
                      Broj_NaN  Procenat_NaN
USMER                        0           0.0
MEDICAL_UNIT                 0           0.0
SEX                          0           0.0
PATIENT_TYPE                 0           0.0
DATE_DIED                    0           0.0
INTUBED                      0           0.0
PNEUMONIA                    0           0.0
AGE                          0           0.0
PREGNANT                     0           0.0
DIABETES                     0           0.0
COPD                         0           0.0
ASTHMA                       0           0.0
INMSUPR                      0           0.0
HIPERTENSION                 0           0.0
OTHER_DISEASE                0           0.0
CARDIOVASCULAR               0           0.0
OBESITY                      0           0.0
RENAL_CHRONIC                0           0.0
TOBACCO                      0           0.0
CLASIFFICATION_FINAL         0           0.0
ICU     

In [24]:
# Sada cemo filtrirati pacijente tako da ostanu samo oni za koje je test na COVID oznacen kao sigurno pozitivan 

print(f"Broj redova pre filtriranja: {df.shape[0]}")

df = df[df['CLASIFFICATION_FINAL'] <= 3].copy()

print(f"Broj redova nakon zadržavanja samo COVID pozitivnih: {df.shape[0]}")


Broj redova pre filtriranja: 1022839
Broj redova nakon zadržavanja samo COVID pozitivnih: 387382


# KREIRANJE CILJNIH PROMENLJIVIH I FEATURE ENGENEERING NA OSNOVU ISTRAŽIVAČNIH PITANJA

Prvo istraživačko pitanje

In [25]:
# Kreiranje ciljne promenljive DEATH
df['DEATH'] = np.where(df['DATE_DIED'] == '9999-99-99', 0, 1)

# Kreiranje starosnih grupa
bins = [0, 19, 39, 59, 79, 120]
labels = ['0-19', '20-39', '40-59', '60-79', '80+']

df['AGE_GROUP'] = pd.cut(df['AGE'], bins=bins, labels=labels, right=True, include_lowest=True)

# Procenat smrtnosti po starosnim grupama
mortality_summary = df.groupby('AGE_GROUP', observed=False)['DEATH'].mean() * 100
print("Procenat smrtnosti po starosnim grupama:")
print(mortality_summary)

# Prikaz smrtnosti po polu
gender_mortality = df.groupby(['SEX', 'DEATH']).size().unstack()
print("\nDistribucija po polu i ishodu:")
print(gender_mortality)

Procenat smrtnosti po starosnim grupama:
AGE_GROUP
0-19      1.568190
20-39     2.188098
40-59    12.109798
60-79    37.619547
80+      52.975888
Name: DEATH, dtype: float64

Distribucija po polu i ishodu:
DEATH       0      1
SEX                 
1      162435  18264
2      172771  33912


In [26]:
# Lista kolona koje predstavljaju hronične bolesti, tj. naše komorbitete
# po dokumentacijji, vrednost 1 je DA, tj. prisustvo bolesti, 2 je NE
chronic_deseases = ['PNEUMONIA', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR', 
                'HIPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 
                'OBESITY', 'RENAL_CHRONIC']

# Konvertovaćemo 2 u 0 kako bismo mogli da ih saberemo hronične bolesti
temp_cm_df = df[chronic_deseases].replace(2, 0)
df['COMORBIDITIES_COUNT'] = temp_cm_df.sum(axis=1)

# Grupisaćemo ih prema brojnosti -> 0, 1, 2, 3+
def classify_comorbidity(n):
    if n == 0: return '0'
    if n == 1: return '1'
    if n == 2: return '2'
    return '3+'

df['COMORBIDITY_GROUP'] = df['COMORBIDITIES_COUNT'].apply(classify_comorbidity)

# Koliko ljudi iz svake grupe je završilo na ICU (1=DA, 2=NE)
icu_by_comorbidity = pd.crosstab(df['COMORBIDITY_GROUP'], df['ICU'])
print("\nTabela:      Broj bolesti vs ICU")
print(icu_by_comorbidity)

# Ako pacijent ima preko 60 godina i bar jedan komorbiditet, onda je on visokorizican
df['HIGH_RISK'] = ((df['AGE'] > 60) & (df['COMORBIDITIES_COUNT'] > 0)).astype(int)

df['PNEUMONIA_IN_ELDERLY'] = ((df['PNEUMONIA'] == 1) & (df['AGE'] > 60)).astype(int)


Tabela:      Broj bolesti vs ICU
ICU                 1.0     2.0
COMORBIDITY_GROUP              
0                   555  193461
1                  3298   98390
2                  3178   49975
3+                 3186   35339


In [27]:
#Pripreme za Spark
# 1. Pretvaranje svih 2 (NE) u 0 (NE) za ML
binary_cols = ['PNEUMONIA', 'SEX', 'PREGNANT', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR', 
               'HIPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY', 
               'RENAL_CHRONIC', 'TOBACCO', 'ICU', 'INTUBED']

for col in binary_cols:
    df[col] = df[col].replace({2: 0})

# Patient type cemo radi lakse citljivosti prevesti u 'HOSPITALIZED', gde je pozitivna klasa potrebno da bude 'hospitalizovan' - 1
df['HOSPITALIZED'] = df['PATIENT_TYPE'].replace({1: 0, 2: 1})

In [28]:
# Obrisacemo DATE_DIED jer smo to vec iskoristili za kreiranje atributa DEATH, pa nam nece trebati vise
# Classification_final zapravo sada znaci da pacijent ima covid, tako da je brisemo
# Patient type nam je sada Hospitalized zapravo

df.drop(columns=['CLASIFFICATION_FINAL', 'DATE_DIED', 'PATIENT_TYPE'], inplace=True)


In [29]:
df.head(15)

,USMER,MEDICAL_UNIT,SEX,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,COPD,ASTHMA,...,RENAL_CHRONIC,TOBACCO,ICU,DEATH,AGE_GROUP,COMORBIDITIES_COUNT,COMORBIDITY_GROUP,HIGH_RISK,PNEUMONIA_IN_ELDERLY,HOSPITALIZED
0,2,1,1,0.0,1.0,65,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1,60-79,2.0,2,1,1,0
2,2,1,0,1.0,0.0,55,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1,40-59,1.0,1,0,0,1
4,2,1,0,0.0,0.0,68,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1,60-79,2.0,2,1,0,0
5,2,1,1,0.0,1.0,40,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,40-59,1.0,1,0,0,1
6,2,1,1,0.0,0.0,64,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,60-79,0.0,0,0,0,0
7,2,1,1,0.0,1.0,64,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0,60-79,5.0,3+,1,1,0
8,2,1,1,0.0,0.0,37,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0,20-39,3.0,3+,0,0,1
9,2,1,1,0.0,0.0,25,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,20-39,0.0,0,0,0,1
10,2,1,1,0.0,0.0,38,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,20-39,0.0,0,0,0,0
11,2,1,0,0.0,0.0,24,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,20-39,0.0,0,0,0,1


In [30]:
df.to_csv('../data/Covid_Data_Clean.csv', index=False)
print("Sacuvali smo uspesno ociscene podatke")

Sacuvali smo uspesno ociscene podatke
